# Introduction

Previously, we looked at fungal leucine tRNAs in GtRNAdb. Next, we're going to look at all eukaryotic leucine tRNAs in GtRNAdb. This will involve two extra steps. I'll need to build a phylogenetic tree, which will be used for autodetection of identity elements. This will replace the hardcoded yeast IDEs. 

We've learned that in fungi, the lower bound for _percentage of tRNAs missing an IDE_ is 20% in a single species, while every single species contained at least one tRNA with the proper IDEs. So for autodetection of IDEs, what can we set as a cutoff? What if all but one species has the IDEs? This could mean that the one species has a mutated synthetase. We need to set a boundary for the amount of evidence that will convince us that a conserved position is an identity element. I'd say that if 80% of species within a clade have the IDE in at least one tRNA, then it's convincing. 

The workflow is as follows:
1. Generate multi-tiered phylogenetic tree
2. Align and filter tRNAs from each clade
3. For each clade, find and score potential IDEs.
    - This includes base pairs and single bases.
    - Scoring will need to take into account a) number of species with at least one tRNA with the IDE, and b) percent of tRNAs with the IDE within each species. (a) is a scalar, while (b) is a vector. So we'll take the product of (a) and the mean of (b), for a score between 0 and 1. Let $s$ be the number of subclades in the clade, while $s_i$ is the subset of subclades containing at least one tRNA with an IDE $i$. $s$ can refer to species (e.g. _S. cerevisiae_ within fungi) or clades (e.g. fungi within eukaryotes). Let $t_s$ be the number of tRNAs within a subclade $s$, and $t_i$ be the number of tRNAs containing an IDE $i$. Then: 
$$\text{Score} = \frac{s_i}{s}\frac{\sum^s{t_{s,i}}}{s}$$
    - As a heuristic, we will not score any positions with $s_i < 0.8$.
4. Imagine a phylogenetic tree with root R branching into clades A, B, and C. If steps #2-#4 are performed for each of A, B, and C, we will now want to perform a R-wide analysis of the IDEs identified within subclades A, B, and C. Since this IDE analysis is entirely computational, some phylogenetic context is necessary. Then, we will want to perform a t-test for the scores of each IDE, between R and A (or B or C). The null hypothesis is that they are the same, e.g. conserved. 
    - Alternatively, we can assume that positions without evolutionary pressure have scores that follow a normal distribution, with a mean of $0.25 \cdot 0.25 = 0.0625$. But tRNAs are subject to a variety of compounding pressures, and have different evolutionary timelines, so actually, this is a silly assumption.
    - Either way, we will need to apply multiple testing corrections.